In [ ]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import ast
pd.set_option('display.max_columns', None)

# **ETL de Archivos YELP**
---

### **checkin.json**
---

**Carga de lo datos**

In [ ]:
df_checkin_yelp=pd.read_json(r'/content/drive/MyDrive/P_F_H/checkin.json',lines=True)
df_checkin_yelp.head(2)
# Archivo con datos que posiblemente no se usen

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."


**Buscamos elementos nulos**

In [ ]:
df_checkin_yelp.isnull().sum()

business_id    0
date           0
dtype: int64

**Revisamos si tiene duplicados**

In [ ]:
df_checkin_yelp.duplicated().sum()

0

**Obtenemos Fecha y hora**

In [ ]:
desanidados = []

for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        time, date = date.split(' ')
        desanidados.append([business_id, time, date])

df_desanidado = pd.DataFrame(desanidados, columns=['business_id', 'hour', 'fecha'])

**Cargamos datos en tabla checking la cual se puede usar para una analisis alternativo**

In [ ]:
df_desanidado.to_csv('checkin.csv', index=False)

### **user.parquet**
---

**Extraccion de los datos(desde la fuente)**

In [ ]:
df_y1=pd.read_parquet('/content/drive/MyDrive/P_F_H/user_y.parquet')
df_y1.head(2)

,user_id,review_count,yelping_since,elite,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,585,2007-01-25 16:47:26,2007,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,4333,2009-01-25 04:35:42,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",3.74


**Transformacion de los datos**

**Busqueda valores nulos**

In [ ]:
df_y1.isnull().sum()

**Realizamos un filtro de los usuarios segun cantidad de reviews que realizaron  y si estas fueron consideradas utiles**

In [ ]:
df_y1[(df_y1['review_count']>=10) & (df_y1['useful']>=10)]

**Eliminamos duplicados**

In [ ]:
#Conteo de duplicados
df_y1.duplicated().sum()

In [ ]:
df_y1.drop_duplicates(inplace=True)

**Eliminamos columnas que no se usaran**

In [ ]:
df_y1.drop(['name','compliment_hot','compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos'], axis=1,inplace=True)

In [ ]:
df_y1.drop(['useful','funny','cool','fans','friends'], axis=1,inplace=True)

**Aplicamos transformaciones a los tipos de datos**

In [ ]:
#cambiamos a tipo datetime la variable yelping_since
df_y1['yelping_since'] = pd.to_datetime(df_y1['yelping_since'])

#Creamos una columna año a partir de elite que devuelva una lista de años separado por comas
df_y1['years'] = df_y1['elite'].str.split(',')

In [ ]:
#reemplazamos valores vacios por np.nan en years
df_y1['years'] = df_y1['years'].replace("", np.nan)

#reemplazamos 20 po 2020 en years
df_y1['years'] = df_y1['years'].replace("20", "2020")

**Carga de datos resultante**

In [ ]:

# archivo con los usuario mas votados o reviews significativas
df_y1.to_parquet('user_y.parquet', index=False, compression='gzip')

**Eliminamos las columnas elite y years del dataframe original user.parquet**

In [ ]:
df_User_yelp.drop(columns=['elite'], inplace=True)
df_User_yelp.drop(columns=['years'], inplace=True)

###**business.pkl**
---

**Carga de datos**

In [ ]:
df_business= pd.read_pickle('/content/drive/MyDrive/P_F_H/business.pkl')

**Se agregan nuevas columnas al dataframe**

In [ ]:
df_business['NAME']=None
df_business['REVIEW_COUNT']=None
df_business['POSTAL_CODE']=None
df_business['CITY']=None
df_business['STATE']=None
df_business['BUSINESS_ID']=None
df_business['ADDRESS']=None
df_business['LATITUDE']=None
df_business['LONGITUDE']=None
df_business['STARS']=None
df_business['IS_OPEN']=None
df_business['ATTRIBUTES']=None
df_business['CATEGORIES']=None
df_business['HOURS']=None

In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 42 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

**Creamos la columna "NAME" del DataFrame df_business extrayendo los datos y agregando .**



In [ ]:
for index,i in enumerate(df_business.name.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['NAME'][index]=''.join(arr)

In [ ]:
df_business.NAME

0         Abby Rappoport, LAC, CMQ
1                    The UPS Store
2                           Target
3               St Honore Pastries
4         Perkiomen Valley Brewery
                    ...           
150341                Binh's Nails
150342        Wild Birds Unlimited
150343           Claire's Boutique
150344    Cyclery & Fitness Center
150345                     Sic Ink
Name: NAME, Length: 150346, dtype: object

**Creamos la columna 'CITY' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.city.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CITY'][index]=''.join(arr)

**Creamos la columna 'STATE' del DataFrame df_business extrayendo los datos y agregando .**


In [ ]:
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['STATE'][index]=''.join(arr)

**Creamos la columna 'REVIEW_COUNT' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.review_count.values):
    arr=[]
    for e in i:
        if isinstance(e,int):
         arr.append(e)
    df_business['REVIEW_COUNT'][index]=arr[0]

**Creamos la columna 'BUSINESS_ID' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.business_id.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['BUSINESS_ID'][index]=''.join(arr)


**Creamos la columna 'ADDRESS' del DataFrame df_business extrayendo los datos y agregando .**



In [ ]:
for index,i in enumerate(df_business.address.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['ADDRESS'][index]=''.join(arr)

**Creamos la columna 'POSTAL_CODE' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.postal_code.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['POSTAL_CODE'][index]=''.join(arr)

**Creamos la columna 'LATITUDE' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.latitude.values):
    arr=[]
    for e in i:
       if e>1:
         arr.append(e)
    df_business['LATITUDE'][index]=arr[0]

**Creamos la columna 'LONGITUDE' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.longitude.values):
    arr=[]
    for e in i:
        if e<-1:
            arr.append(e)
    df_business['LONGITUDE'][index]=arr[0]

**Creamos la columna 'STARS' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.stars.values):
    arr=[]
    for e in i:
       if e>0.1:
         arr.append(e)
    df_business['STARS'][index]=arr[0]

**Creamos la columna 'IS_OPEN' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.is_open.values):
    arr=[]
    for e in i:
       if e >=0:
         arr.append(e)
    df_business['IS_OPEN'][index]=arr[0]

**Creamos la columna 'ATRIBUTES' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.attributes.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:
     df_business['ATTRIBUTES'][index]=arr[0]

**Creamos la columna 'CATEGORIES' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
# AGREGA LAS CADENAS OBTENIDAS DE CATEGORIES EN UNA
for index,i in enumerate(df_business.categories.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CATEGORIES'][index]=''.join(arr)

**Creamos la columna 'HOURS' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.hours.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:
     df_business['HOURS'][index]=arr[0]

In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 42 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

**Asigno al dataframe  solo a las columnas nuevas**

In [ ]:
df_business=df_business[['BUSINESS_ID','NAME','REVIEW_COUNT','CITY','STATE','ADDRESS','POSTAL_CODE','LATITUDE','LONGITUDE','STARS','IS_OPEN','ATTRIBUTES','CATEGORIES','HOURS']]

In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BUSINESS_ID   150346 non-null  object
 1   NAME          150346 non-null  object
 2   REVIEW_COUNT  150346 non-null  object
 3   CITY          150346 non-null  object
 4   STATE         150346 non-null  object
 5   ADDRESS       150346 non-null  object
 6   POSTAL_CODE   150346 non-null  object
 7   LATITUDE      150346 non-null  object
 8   LONGITUDE     150346 non-null  object
 9   STARS         150346 non-null  object
 10  IS_OPEN       150346 non-null  object
 11  ATTRIBUTES    136602 non-null  object
 12  CATEGORIES    150346 non-null  object
 13  HOURS         127123 non-null  object
dtypes: object(14)
memory usage: 21.2+ MB


In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BUSINESS_ID   150346 non-null  object
 1   NAME          150346 non-null  object
 2   REVIEW_COUNT  150346 non-null  object
 3   CITY          150346 non-null  object
 4   STATE         150346 non-null  object
 5   ADDRESS       150346 non-null  object
 6   POSTAL_CODE   150346 non-null  object
 7   LATITUDE      150346 non-null  object
 8   LONGITUDE     150346 non-null  object
 9   STARS         150346 non-null  object
 10  IS_OPEN       150346 non-null  object
 11  ATTRIBUTES    136602 non-null  object
 12  CATEGORIES    150346 non-null  object
 13  HOURS         127123 non-null  object
dtypes: object(14)
memory usage: 21.2+ MB


In [ ]:
df_business=df_business.rename(columns={'BUSINESS_ID':'business_id'})
df_business=df_business.rename(columns={'NAME':'name'})
df_business=df_business.rename(columns={'REVIEW_COUNT':'review_count'})
df_business=df_business.rename(columns={'CITY':'city'})
df_business=df_business.rename(columns={'STATE':'state'})
df_business=df_business.rename(columns={'ADDRESS':'address'})
df_business=df_business.rename(columns={'POSTAL_CODE':'postal_code'})
df_business=df_business.rename(columns={'LATITUDE':'lalitude'})
df_business=df_business.rename(columns={'LONGITUDE':'longitude'})
df_business=df_business.rename(columns={'STARS':'stars'})
df_business=df_business.rename(columns={'IS_OPEN':'is_open'})
df_business=df_business.rename(columns={'ATTRIBUTES':'attributes'})
df_business=df_business.rename(columns={'CATEGORIES':'categories'})
df_business=df_business.rename(columns={'HOURS':'hours'})

In [ ]:
df_business.head(3)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",7,Santa Barbara,,"1616 Chapala St, Ste 2",93101,34.426679,-119.711197,5.0,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,15,Affton,,87 Grasso Plaza Shopping Center,63123,38.551126,-90.335695,3.0,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,22,Tucson,,5255 E Broadway Blvd,85711,32.223236,-110.880452,3.5,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


In [ ]:
df_business.state

0           
1           
2           
3         CA
4         MO
          ..
150341    IN
150342    DE
150343    AB
150344    AB
150345    TN
Name: state, Length: 150346, dtype: object

In [ ]:
df_business.to_parquet('BUSINESS_YELP.parquet', index=False, compression='gzip')

In [ ]:
df_business.head(1)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",7,Santa Barbara,,"1616 Chapala St, Ste 2",93101,34.426679,-119.711197,5.0,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None


In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   review_count  150346 non-null  object
 3   city          150346 non-null  object
 4   state         150346 non-null  object
 5   address       150346 non-null  object
 6   postal_code   150346 non-null  object
 7   lalitude      150346 non-null  object
 8   longitude     150346 non-null  object
 9   stars         150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150346 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 21.2+ MB


**Filtramos por el estado de New York**

In [ ]:
df_business[df_business['STATE']=='NJ'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8535 entries, 45 to 150340
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BUSINESS_ID   8535 non-null   object
 1   NAME          8535 non-null   object
 2   REVIEW_COUNT  8535 non-null   object
 3   CITY          8535 non-null   object
 4   STATE         8535 non-null   object
 5   ADDRESS       8535 non-null   object
 6   POSTAL_CODE   8535 non-null   object
 7   LATITUDE      8535 non-null   object
 8   LONGITUDE     8535 non-null   object
 9   STARS         8535 non-null   object
 10  IS_OPEN       8535 non-null   object
 11  ATTRIBUTES    7783 non-null   object
 12  CATEGORIES    8535 non-null   object
 13  HOURS         7220 non-null   object
dtypes: object(14)
memory usage: 1000.2+ KB


In [ ]:
df_NYork=df_business[df_business['state']=='NJ'].copy()
df_NYork.head(2)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
45,wghnIlMb_i5U46HMBGx9ig,China Dragon Restaurant,23,Tucson,NJ,"1625 W Valencia Rd, Ste 101-103",85746,32.132305,-110.999985,3.0,0,"{'WiFi': ''no'', 'BusinessParking': '{'garage'...","Restaurants, Chinese","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
76,-4dYswJy7SPcbcERvitmIg,Pathmark,34,Philadelphia,NJ,3021 Grays Ferry Ave,19146,39.940403,-75.193297,2.5,0,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Grocery","{'Monday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'W..."


In [ ]:
df_NYork[df_NYork['name']=="McDonald's"].business_id.count()

50

In [ ]:
df_NYork[df_NYork['name']=="McDonald's"].head(3)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
361,aNXw3PkXVt8ANwLyCfcmpg,McDonald's,17,Lansdale,NJ,2333 Welsh Rd,19446,40.263706,-75.317916,1.5,1,"{'WiFi': 'u'free'', 'GoodForKids': 'True', 'Bu...","Restaurants, Fast Food, Burgers, Food, Coffee ...","{'Monday': '6:0-23:0', 'Tuesday': '6:0-23:0', ..."
3161,CxoRg-ZcXdpqXWmUJliYog,McDonald's,28,Hudson,NJ,9441 St Rd 52,34669,28.331619,-82.658409,2.5,1,"{'HasTV': 'True', 'Alcohol': 'u'none'', 'Busin...","Restaurants, Food, Fast Food, Coffee & Tea, Bu...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
3593,2gTnSpYBmg_5JubGpCtSQQ,McDonald's,41,Fishers,NJ,9611 N By Ne Blvd,46038,39.927626,-86.025654,2.0,1,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...","Coffee & Tea, Food, Burgers, Fast Food, Restau...","{'Monday': '5:0-1:0', 'Tuesday': '5:0-1:0', 'W..."


**Realizamos converciones en los tipos de dato nesesarias**

In [ ]:
#CONVERSIONES A REALIZAR AL FINAL DE EL TRATAMIENTO DE TODOS LOS DATOS
df_NYork["business_id"] = df_NYork["business_id"].astype('str')
df_NYork["name"] = df_NYork["name"].astype('str')
df_NYork["review_count"] = df_NYork["review_count"].astype('int')
df_NYork["city"] = df_NYork["city"].astype('str')
df_NYork["state"] = df_NYork["state"].astype('str')
df_NYork["address"] = df_NYork["address"].astype('str')
df_NYork["postal_code"] = df_NYork["postal_code"].apply(pd.to_numeric, errors='coerce')
df_NYork["latitude"] = df_NYork["latitude"].apply(pd.to_numeric, errors='coerce')
df_NYork["longitude"] = df_NYork["longitude"].apply(pd.to_numeric, errors='coerce')
df_NYork["stars"] = df_NYork["stars"].apply(pd.to_numeric, errors='coerce')


In [ ]:
df_NYork.to_parquet('BUSINESS_YELP_YORK.parquet', index=False, compression='gzip')

**Generamos un nuevo dataframe con 'business_id',
'categories'**

In [ ]:
#Tabla de dimension de categorias
df_cat = df_NYork.loc[:, ['business_id','categories']]
df_cat.head(3)

,business_id,categories
45,wghnIlMb_i5U46HMBGx9ig,"Restaurants, Chinese"
76,-4dYswJy7SPcbcERvitmIg,"Food, Grocery"
86,sE6jSnvMts_MAn-b4OkMAw,"Pet Groomers, Pet Sitting, Pets, Pet Services"


In [ ]:
# Dividir los valores de la columna "categories" por coma y expandirlos en filas
df_cat['categories'] = df_cat['categories'].str.split(',')
df_cat = df_cat[['business_id', 'categories']].explode('categories')

In [ ]:
#Creamos un array con las categorias unicas
categorias = df_cat["categories"].unique()
#Creamos la tabla de dimension de categorias de yelp
df_categorias = pd.DataFrame(categorias, columns=["Descripcion"])
df_categorias['IdCategoria'] = df_categorias.index

In [ ]:
df_cat

,business_id,categories
45,wghnIlMb_i5U46HMBGx9ig,"Restaurants, Chinese"
76,-4dYswJy7SPcbcERvitmIg,"Food, Grocery"
86,sE6jSnvMts_MAn-b4OkMAw,"Pet Groomers, Pet Sitting, Pets, Pet Services"
91,nUqrF-h9S7myCcvNDecOvw,"Towing, Automotive, Body Shops"
114,4dVrWc4Nhuw9apfk9BWEcA,"Grocery, Food, Shopping, Fashion, Drugstores, ..."
...,...,...
150285,fWeWzB9STxcX40AgSEQVcw,"Parks, Zoos, Active Life, Botanical Gardens, M..."
150302,DMYTGaunth3BMi7SmQEdvQ,"Pet Groomers, Pets, Pet Services, Pet Boarding..."
150311,S-Jn_08NWHhdgryQgzDCFA,"Nail Salons, Beauty & Spas"
150317,Q7JYAMNzI1IpUd2edflmTA,"Men's Hair Salons, Hair Salons, Barbers, Beaut..."


**Carga en la tabla de dimensiones de categorias disponibles de los locales de la franquicia McDonald's en el estado de New York**

In [ ]:
df_categorias.head(5)

In [ ]:
df_categorias.to_csv("Categorias_Yelp.csv")

**Generamos tabla de dimension atributos de locales de franquicia en el estado New York**

In [ ]:
df_atribute = df_NYork.loc[:, ["business_id",'attributes']]

**Dividimos los elementos de la columna 'attributes'**

In [ ]:
for index, i in enumerate(df_atribute['attributes']):
    if isinstance(i,str):
       df_atribute['attributes'][index]= i[1:-1].split(',')

**Expandimos la columna 'attributes' en filas separadas, manteniendo los valores correspondientes de la columna 'business_id'.**

In [ ]:
df_atribute = df_atribute[['business_id', 'attributes']].explode('attributes')

**Se eliminan los corchetes y comillas dobles de la columna 'attributes'**

In [ ]:
df_atribute['attributes'] = df_atribute['attributes'].str.replace('{', '').str.replace('}', '')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('"', '')

<ipython-input-70-467e46ee6ade>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_atribute['attributes'] = df_atribute['attributes'].str.replace('{', '').str.replace('}', '')


In [ ]:
df_atribute.head(5)

,business_id,attributes
45,wghnIlMb_i5U46HMBGx9ig,WiFi
45,wghnIlMb_i5U46HMBGx9ig,BusinessParking
45,wghnIlMb_i5U46HMBGx9ig,HasTV
45,wghnIlMb_i5U46HMBGx9ig,Alcohol
45,wghnIlMb_i5U46HMBGx9ig,RestaurantsDelivery


**Creacion de tabla con business_id mas lista de atributos por negocio en el estado de New York**

In [ ]:
df_atribute.to_csv("BusinessAtributos_Yelp.csv")

**Creacion de DataFrame que contiene los valores únicos de la columna 'attributes', junto con una columna de identificación.**

In [ ]:
atributos = df_atribute["attributes"].unique()
df_atributos = pd.DataFrame(atributos, columns=["Descripcion"])
df_atributos['IdAtributos'] = df_atributos.index

In [ ]:
df_atributos.head(3)

,Descripcion,IdAtributos
0,WiFi,0
1,BusinessParking,1
2,HasTV,2


**Carga de tabla de atributos unicos en los locales listados en el estado de New York**

In [ ]:
df_atributos.to_csv("Atributos_Yelp.csv")

**Creamos la tabla de dimensiones "Hours" a partir de aux**

In [ ]:
df_hours = df_NYork.loc[:, ["business_id",'hours']]

In [ ]:
df_hours.head(3)

,business_id,hours
45,wghnIlMb_i5U46HMBGx9ig,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
76,-4dYswJy7SPcbcERvitmIg,"{'Monday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'W..."
86,sE6jSnvMts_MAn-b4OkMAw,"{'Monday': '8:0-16:0', 'Tuesday': '8:0-16:0', ..."


**Exportamos la tabla de Hours**

In [ ]:
df_hours.to_csv("Datasets/BusinessHoras.csv")

**Creamos el dataframe df_Hour_detalle**

In [ ]:
df_hour_detalle=df_hours[['business_id','hours']]

**Desglosamos los diferentes horarios en diferentes filas para cada valor de forma individual**

In [ ]:
df_hour_detalle=df_hour_detalle[['business_id','hours']].explode('hours')

**Creamos un dataframe llamado hour con las horas unicas de df_hour_detalle**

In [ ]:
hours=df_hour_detalle['hours'].unique()

**creacion de  DataFrame que contiene  "Descripcion" (valores de la columna 'hours' )**

In [ ]:
DescHour=pd.DataFrame(hours,columns=["Descripcion"])
DescHour['id_hour']=DescHour.index

In [ ]:
DescHour

,Descripcion,id_hour
0,Monday,0
1,Tuesday,1
2,Wednesday,2
3,Thursday,3
4,Friday,4
5,Saturday,5
6,Sunday,6
7,None,7


**Cargamos en tabla de dimension Horarios**

In [ ]:
DescHour.to_csv('Horarios_Yelp.csv',index=False)

**Hacemos una union entre df_hour_detalle y DescHour**

In [ ]:
df_hour_detalle.merge(DescHour,left_on='hours',right_on='Descripcion',how='inner')[['business_id','id_hour']]


,business_id,id_hour
0,wghnIlMb_i5U46HMBGx9ig,0
1,-4dYswJy7SPcbcERvitmIg,0
2,sE6jSnvMts_MAn-b4OkMAw,0
3,nUqrF-h9S7myCcvNDecOvw,0
4,4dVrWc4Nhuw9apfk9BWEcA,0
...,...,...
46822,76vbrj2OZhgPJUv46Doe5Q,7
46823,yDPwHWr9bY88DhoQmMPWmg,7
46824,c6RWw740cIB9-efs9tSLyg,7
46825,oAsfYTJE3cOJXk7Dmb-1TQ,7


**Cargamos en tabla de dimension Horarios por Negocio**

In [ ]:
df_hour_detalle.to_csv('Business_Horas_Yelp.csv',index=False)

**Creamos el dataframe df_detalleatributo a partir de la union entre df_atribute y df_atributos**

In [ ]:
df_detalleatributo = df_atribute.merge(df_atributos, left_on="attributes", right_on="Descripcion", how="inner")

In [ ]:
df_detalleatributo.head(5)

,business_id,attributes,Descripcion,IdAtributos
0,wghnIlMb_i5U46HMBGx9ig,WiFi,WiFi,0
1,nUqrF-h9S7myCcvNDecOvw,WiFi,WiFi,0
2,9C2rpb56aQvW0ViZHK9sPw,WiFi,WiFi,0
3,2oxrOO3c9_mQmqM9kwEm9Q,WiFi,WiFi,0
4,QZU7TcrztBb3tXaPbVCkXg,WiFi,WiFi,0


**Eliminamos las columnas "attributes" y "Descripcion" de df_detalleatributo**

In [ ]:
df_detalleatributo.drop(columns = "attributes",inplace=True)
df_detalleatributo.drop(columns = "Descripcion",inplace=True)

In [ ]:
df_detalleatributo.tail(5)

,business_id,IdAtributos
69580,ccNH_y1-c58tX74o4V4mlw,37
69581,bNSBYd-wASjWZ3hZWb0sGQ,38
69582,lvi4u_XVzvL06EAj0hUmfw,38
69583,SvA2IQ9SyI1Dh7dtwNNs3A,38
69584,K2MrnWrEEg9FzHrm9lve8A,38


**Carga en tabla detalles de Atributo por Negocio**

In [ ]:
df_detalleatributo.to_csv("Business_Atributos_Yelp.csv")

### **tip.json**
---

**Abrimos el archivo json**

In [ ]:
json_objects=[]

with open('../tip.json', 'r',encoding='utf-8') as f:
    for line in f:
        json_objects.append(json.loads(line))


df_tip = pd.DataFrame(json_objects)

In [ ]:
df_tip=pd.read_json(r'/content/drive/MyDrive/P_F_H/tip.json',lines=True)
df_tip.head(6)
# archivo a entrecruzar por id de usuario

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
5,trf3Qcz8qvCDKXiTgjUcEg,7Rm9Ba50bw23KTA8RedZYg,"Chili Cup + Single Cheeseburger with onion, pi...",2012-03-13 04:00:52,0


In [ ]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


 **Obtenemos un DataFrame que contenga solo las filas correspondientes a los negocios presentes en BusinessYelp**

In [ ]:
df_tip=df_tip[df_tip['business_id'].isin(df_NYork.business_id.unique().tolist())]

In [ ]:
df_tip

,user_id,business_id,text,date,compliment_count
9,4ay-fdVks5WMerYL_htkGQ,pICJRcyqW1cF96Q3XhLSbw,Starbucks substitute in boring downtown Tampa....,2012-06-09 22:57:04,0
38,_YBX0Fy9pI0ktOQ9RtFhAg,nzqx3NJ1Yf-1bugLNikSCg,Hours are incorrect on Yelp. They close at 3PM...,2016-07-30 20:44:04,0
42,VECuzTUvA3FvaqSUrdXIMg,TghRoAMx43V-9l7mH-SENg,Best Mahi fish tacos blackened I have ever had...,2016-04-18 23:24:33,0
51,g-jD73ukXg5c3IKpv_sTyw,HgPTy_OGoaxllb2EXkC1sQ,"Good peruvian place! For $10 a place, its wort...",2013-05-23 16:34:31,0
64,cogPv5baWhsR-_Dcw3-Taw,-Xld662dL8WxrwP--bclIQ,The yum yum roll is delicious!,2011-05-01 01:48:56,0
...,...,...,...,...,...
908797,vUZz4qse2KQS9g8ux2ZIug,R_eXVjWKjlrf_HjJpmFmdg,The carnitas were dry and bland... the chips a...,2021-09-18 21:02:17,0
908814,P8Q5y_pDMkH72bQ_mW0buA,LnTZm3Lvt_aPWzpd61eq6g,Very disappointing service. Very slow and staf...,2014-07-27 11:27:36,0
908873,zCXrIBe0LhylkQW_1VA0SA,GXFMD0Z4jEVZBCsbPf4CTQ,Excellent,2018-03-12 22:31:00,0
908879,UDaA2IhjcCe3IAmPGz8w1g,GXFMD0Z4jEVZBCsbPf4CTQ,Great food. Well worth the 90 minute wait.,2019-04-06 05:23:27,0


**Hacemos una unión df_tip con df_NYork**

In [ ]:
df_tip=df_NYork.merge(df_tip,left_on='business_id',right_on='business_id',how='right')[['user_id', 'business_id', 'text', 'date', 'compliment_count']]

**Hacemos una unión df_tip con UsuarioYelp**

In [ ]:
df_tip=dfusuario.merge(df_tip,left_on='yelp_id',right_on='user_id',how='right')[['id_user', 'business_id', 'text', 'date', 'compliment_count']]

**Cargamos dato resultante en tabla tip de Yelp**

In [ ]:
df_tip.to_csv('tipsYelp.csv',index=False)

### **review.json**

**Leemos el archivo review.json de Yelp**

In [ ]:
jsonarr=[]
with open("/content/drive/MyDrive/P_F_H/review.json", 'r',encoding='utf-8') as f:
    for line in f:
        jsonarr.append(json.loads(line))


df_review = pd.DataFrame(jsonarr)

In [ ]:
df = pd.read_json("/content/drive/MyDrive/P_F_H/review.json", orient ='index')
print(df)

# **ETL de Archivos de Google Maps**
---

## **Review Estados**
---

### **Estado New York**
---